In [63]:
import gurobipy as gp
from gurobipy import *
import random as rd
import numpy as np

In [64]:
import openpyxl
wb1 = openpyxl.load_workbook('South2.xlsx', data_only = True)
st = wb1.sheetnames
st1 = []

Pl = []
for sheet_name in st:
    worksheet1 = wb1[sheet_name]
    st1.append(sheet_name)
    
    g1_value = worksheet1['G1'].value
    Pl.append(g1_value)
    

#print(stations)
print(Pl)
print(st1)

[9, 7, 7, 5, 7, 7, 2, 6, 4, 7]
['Vizianagaram Junction', 'Visakhapatnam Junction', 'Rajahmundry', 'Ongole', 'Kadapa', 'Guntakal Jn', 'Eluru', 'tirupati', 'Bhimavara Town', 'Anantapur']


In [65]:
Routes=['North', 'South']
NoS=[9, len(st1)]

r = input('Choose a route between North and South: ')
#print(r)
for i in range(len(NoS)):
    if int(Routes.index(r))==i:
        S=NoS[i]
        print('Total number of stations in the route: ', S)
TD=[]
for (i) in range (S):
    d=int(input('\nNumber of days you want to spend in Station '+ st1[i] +'\n'))
    TD.append(d)
TD

Choose a route between North and South: South
Total number of stations in the route:  10

Number of days you want to spend in Station Vizianagaram Junction
2

Number of days you want to spend in Station Visakhapatnam Junction
2

Number of days you want to spend in Station Rajahmundry
2

Number of days you want to spend in Station Ongole
2

Number of days you want to spend in Station Kadapa
2

Number of days you want to spend in Station Guntakal Jn
2

Number of days you want to spend in Station Eluru
2

Number of days you want to spend in Station tirupati
2

Number of days you want to spend in Station Bhimavara Town
2

Number of days you want to spend in Station Anantapur
2


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [66]:
def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)
  return subsets

In [68]:
for i in range(S):
    print("station:  " + str(i+1))
    print(Pl[i]) #number of places in ith station
    L = [p for p in range(Pl[i])]
    print(L)
    T = [b for b in range(TD[i])]
    p = Pl[i]
    d = TD[i]
    
    #place to place travel time
    for j in L:
        for i in L:
            tp = np.random.randint(low=1, high=4, size=(p,p))
            tp = np.triu(tp, 1) + np.triu(tp, 1).T
            np.fill_diagonal(tp, 0)
    print(tp)

    #time of stay per station
    ts = [rd.randint(1, 3) for i in range(p)]  # generate the random vector
    print(ts)
    m=gp.Model('Travel')
    
    #Variable
    xijt = m.addVars(L, L, T, vtype=GRB.BINARY, name ='xijt')
    
    #Objective Function
    obj_fn = sum(xijt[i,j,t] for t in T for i in L for j in L)
    m.setObjective(obj_fn, GRB.MAXIMIZE)
    
    
    #final constrints:  
    m.addConstrs((gp.quicksum(xijt[i,j,t] for t in T) <= 1 for i in L for j in L if i != 0 or j != 0))
    m.addConstrs(gp.quicksum(xijt[i,j,t] for t in T for i in L) <= 1 for j in L if i != 0 or j != 0)
    m.addConstrs(gp.quicksum(xijt[i,j,t] for t in T for j in L) <= 1 for i in L if i != 0 or j != 0)
    m.addConstrs(gp.quicksum(xijt[i,j,t] for i in L) == gp.quicksum(xijt[j,i,t] for i in L) for j in L for t in T)
    m.addConstrs(gp.quicksum(xijt[i,i,t] for t in T) == 0 for i in L)
    m.addConstrs(gp.quicksum(xijt[i,j,t]*tp[i][j] for i in L for j in L) + gp.quicksum(xijt[i,j,t]*ts[j] for j in L) <= 12 for t in T)
    #m.addConstrs(gp.quicksum(xijt[0,j,t] for t in T) <= d for j in L)
    #m.addConstrs(gp.quicksum(xijt[i,0,t] for t in T) <= d for i in L)
    
    m.optimize()
    m.printAttr('X')
    #end
#     for r in range(2, Pl[i]):
#         for subset in itertools.combinations(range(1, Pl[i]), r):
#             # create a binary variable for each subset
#             sub_vars = [xijt[i, j, t] for i in subset for j in subset if i != j for t in T]
#             m.addConstr(gp.quicksum(sub_vars) >= 1)

    
    
    
    #subtour elimination
#     for Su in powerset(L):
#         #print(S)
#         if len(Su)>=1 and (len(Su)<Pl[i]):
#             sumt=0
#             for t in T:
#                 for i in Su:
#                     for j in L:
#                         if j not in Su:
#                             sumt=sumt+xijt[i,j,t]
#                             m.addConstr(sumt, GRB.GREATER_EQUAL, 1)

    #print(xijt[1,5,0])
    
#     print(xijt)

station:  1
9
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[[0 1 3 1 2 2 2 2 2]
 [1 0 2 1 1 2 2 3 1]
 [3 2 0 2 1 2 2 1 1]
 [1 1 2 0 3 2 1 3 1]
 [2 1 1 3 0 3 2 3 3]
 [2 2 2 2 3 0 1 1 1]
 [2 2 2 1 2 1 0 1 2]
 [2 3 1 3 3 1 1 0 1]
 [2 1 1 1 3 1 2 1 0]]
[2, 2, 1, 1, 2, 2, 3, 3, 3]


TypeError: 'Var' object is not iterable